In [1]:
# pip install websockets pylsl

In [2]:
import asyncio
import websockets
import json
from pylsl import StreamInfo, StreamOutlet

# Create an LSL stream
info = StreamInfo('WebSocketLSL', 'Markers', 1, 0, 'string', 'myuid12345')
outlet = StreamOutlet(info)

disaster_mapping = {
    ("Earthquake-Induced Landslide", "Rumoi"): 1,
    ("Earthquake-Induced Landslide", "Abashiri"): 2,
    ("Earthquake-Induced Landslide", "Hidaka"): 3,
    ("Collapsed Highway Tunnel", "Sorachi"): 4,
    ("Collapsed Highway Tunnel", "Abashiri"): 5,
    ("Power Grid Failure", "Ishikari"): 6,
    ("Power Grid Failure", "Rumoi"): 7,
    ("Flooded Residential Zone", "Tokachi"): 8,
    ("Limited Medical Supplies", "Iburi"): 9,
    ("Blocked Rural Roads", "Shiribeshi"): 10,
    ("Lack of Clean Drinking Water", "Kamikawa"): 11,
    ("Hospital Collapse", "Abashiri"): 12,
    ("Dam Overflow", "Oshima"): 13,
    ("Community Shelters Overcrowded", "Tokachi"): 14,
    ("Disrupted Supply Chains", "Hidaka"): 15,
    ("Malfunctioning Communication Towers", "Tokachi"): 16,
    ("Malfunctioning Communication Towers", "Hidaka"): 17,
    ("Tsunami Aftermath", "Tokachi"): 18,
    ("Damaged Water Treatment Facility", "Rumoi"): 19,
    ("Wildfire Approaching Town", "Kushiro"): 20,
    ("Overcrowded Evacuation Shelters", "Sorachi"): 21,
    ("Bridge Collapse", "Hidaka"): 22,
    ("Broken Gas Pipeline", "Soya"): 23,
    ("Food Shortages in Remote Areas", "Hiyama"): 24,
    ("Volunteer Fatigue", "Tokachi"): 25,
}

def get_disaster_marker(name, region, threat_status):
    key = (name, region)
    if key not in disaster_mapping:
        print(f"Unknown disasters: {name} in {region}")
        return None
    disaster_marker = disaster_mapping[key]
    if threat_status.lower() == "resolution_not_started":
        return disaster_marker * 2
    else:
        return disaster_marker * 2 + 1

def get_resource_marker(team):
    team = team.lower()
    if team == "air":
        return "101"
    elif team == "ground":
        return "102"
    else:
        print(f"Unknown resource team: {team}")
        return None

# Configure LSL Value to send
def get_lsl_marker(data):
    
    # Rounds Start and End
    if "session_events" in data:
        for event in data["session_events"]:
            event_type = event.get("event_type", "")
            if event_type == "SessionEvent":
                # Extract session status
                session_status = event.get("data", {}).get("session_status", "").lower()
                if session_status == "started":
                    marker = "200"
                elif session_status == "ended":
                    marker = "201"
                else:
                    marker = "UNKNOWN_SESSION_STATUS"

    # Disasters
    if "config" in data:
        config = data["config"]
        if "events" in config:
            for event in config["events"]:
                if event.get("event_type") == "DisasterEvent":
                    event_data = event.get("data", {})
                    name = event_data.get("name")
                    region = event_data.get("region_id")
                    threat_status = event.get("threat_status", "resolution_not_started")
                    marker = get_disaster_marker(name, region, threat_status)
                    if marker is None:
                        marker = "UNKNOWN DISASTER"
                    else:
                        marker = str(marker)

    # Resources
    if "config" in data:
        config = data["config"]
        if "resources" in config:
            resources = config["resources"]
            if "deployments" in resources:
                for deployment in resources["deployments"]:
                    team = deployment.get("team", "")
                    marker = get_resource_marker(team)
                    if marker is None:
                        marker = "UNKNOWN DISASTER"
                    else:
                        marker = str(marker)

    return marker

# Accept a variable number of arguments so it works with either 1 or 2 parameters.
async def handle_connection(websocket, *args):
    # If a path is provided, use it; otherwise, default to None
    path = args[0] if args else None
    print(f"Client connected. Path: {path}")

    try:
        async for message in websocket:
            print(f"Received WebSocket message: {message}")

            try:
                data = json.loads(message)
                marker = get_lsl_marker(data)
                if marker is None:
                    print(f"Unknown event: {marker}")
                    marker = "UNKNOWN_EVENT"
                else:
                    marker = str(marker)

                # Push the event to LSL
                outlet.push_sample([marker])
                print(f"Sent to LSL: {marker}")

                # Send acknowledgment back to the client
                await websocket.send(f"LSL Received: {marker}")

            except json.JSONDecodeError:
                print("Error: Received non-JSON message")

    except websockets.exceptions.ConnectionClosed:
        print("Client disconnected")

async def main():
    print("Starting WebSocket Server on ws://localhost:8765")
    async with websockets.serve(handle_connection, "localhost", 8765):
        await asyncio.Future()  # Keep the server running

# Use the current event loop or start a new one if needed.
try:
    loop = asyncio.get_running_loop()
    loop.create_task(main())
    print("WebSocket server running in background...")
except RuntimeError:
    asyncio.run(main())

WebSocket server running in background...


Starting WebSocket Server on ws://localhost:8765
Client connected. Path: None
Received WebSocket message: {
    "session_events": [
        {
            "test_session_id": "89b014959d334717b8b12a913013226f",
            "created_at_user_id": "2025-02-27T17:35:44.734267+00:00#6aefc61f0b944f8d970a2fbf2bff11b5",
            "created_at": "2025-02-27T17:35:44.734267+00:00",
            "data": {
                "duration_seconds": 120,
                "session_status": "started"
            },
            "elapsed_seconds": 0,
            "event_type": "SessionEvent",
            "id": "88db8abaa0f44127af5006b5b186ebf6",
            "user": {
                "id": "6aefc61f0b944f8d970a2fbf2bff11b5",
                "nickname": "Ms. Admin",
                "role": "ADMIN"
            }
        },
        {
            "test_session_id": "89b014959d334717b8b12a913013226f",
            "created_at_user_id": "2025-02-27T17:37:45.128113+00:00#6aefc61f0b944f8d970a2fbf2bff11b5",
            "cre